In [1]:
import pandas as pd
import numpy as np
import matplotlib as plot
import datetime
import openpyxl

In [2]:
# Load Data Frame 1 with the Seed Sheet
df_SeedSheet = pd.read_excel("Indian Stocks Dump.xlsx", "Seed Stock Sheet")

In [3]:
# Load Data Frame 2 with data from Web ( Twitter & MoneyControl)
df_WebData = pd.read_excel("Indian Stocks Dump.xlsx", "Web & Social Data")

In [4]:
# Load Data Frame 3 with additional Stock Parameters
df_AdditionalData = pd.read_excel("Indian Stocks Dump.xlsx", "Additional Stock Metrics")

# Cast Ticker & ISIN to String column types
df_AdditionalData = df_AdditionalData.astype({'Ticker':'string', 'ISIN':'string'})

In [5]:
# Merge the Web Data - df_WebData into the main Seed sheet
df_mergedWebData = pd.merge(df_SeedSheet, df_WebData[
    ['Company',
     'Moneycontrol Link',
     'Website',
     'Email',
     'Twitter Link',
     'Twitter Handle',
     'Twitter Followers',
     'Twitter Posts',
     'Twitter Created Date',
     'Twitter Account Age (Years)']],  on = 'Company', how = 'left')

In [6]:
# Capture all the Duplicated data per sheet
df_duplicate_SeednWebData = df_mergedWebData[df_mergedWebData.duplicated('ISIN')]
df_duplicate_AddData_ISIN = df_AdditionalData[df_AdditionalData.duplicated('ISIN')]
df_duplicate_AddData_Ticker = df_AdditionalData[df_AdditionalData.duplicated('Ticker')]
with pd.ExcelWriter('Duplicate Data.xlsx') as writer:  
    df_duplicate_SeednWebData.to_excel(writer, sheet_name='Seed Data')
    df_duplicate_AddData_ISIN.to_excel(writer, sheet_name='Add Data ISIN')
    df_duplicate_AddData_Ticker.to_excel(writer, sheet_name='Add Data Ticker')

In [7]:
# Drop NaN ISIN values from the Seed Data
list_IndexToRemove = df_mergedWebData.index[df_mergedWebData['ISIN'].isna()]
df_mergedWebData.drop(list_IndexToRemove, 0, inplace = True)

C:\Users\amshuk\AppData\Local\Temp/ipykernel_26092/63721468.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_mergedWebData.drop(list_IndexToRemove, 0, inplace = True)


In [8]:
# Drop duplicate Tickers / ISIN from the Additional Sheet 
df_AdditionalData.drop_duplicates(subset = ['ISIN'], keep = 'first', inplace = True )
# Drop NaN ISIN from the Additional Sheet
list_IndexToRemove_NA = df_AdditionalData.index[df_AdditionalData['ISIN'].isna()]
df_AdditionalData.drop(list_IndexToRemove_NA, 0, inplace = True)

C:\Users\amshuk\AppData\Local\Temp/ipykernel_26092/177159638.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_AdditionalData.drop(list_IndexToRemove_NA, 0, inplace = True)


In [9]:
# Remove the Funds & ETF data ( starts with INF***** instead of INE*** )from the Additional Data Sheet
list_IndexOfNonStocks = df_AdditionalData.index[~df_AdditionalData['ISIN'].str.startswith('INE')]
df_AdditionalData.drop(list_IndexOfNonStocks, 0, inplace=True)
df_AdditionalData

C:\Users\amshuk\AppData\Local\Temp/ipykernel_26092/3586761330.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_AdditionalData.drop(list_IndexOfNonStocks, 0, inplace=True)


,Ticker,ISIN,5Y Avg ROE,5Y Revenue Growth,Promoter Holding,No. of Shareholders,Pledged Promoter Holdings,Rating agency Buy Reco,Volatility,Total Debt,Free Cash Flow,Total Revenue(For last QTR)
0,RELIANCE,INE002A01018,10.627669,12.027323,50.614606,3306662.0,0.000000,21.0,25.405350,270648.00,-79652.00,505153.00
1,TCS,INE467B01029,36.796971,9.886696,72.297047,2474310.0,0.345256,22.0,21.197738,7818.00,36969.00,195772.00
2,HDFCBANK,INE040A01034,17.295561,14.242193,25.780743,2151630.0,0.000000,35.0,25.635825,NaN,40780.31,167695.40
3,INFY,INE009A01021,25.828780,11.618708,13.114253,2128827.0,0.000000,36.0,23.936476,5474.00,21724.00,123936.00
4,ICICIBANK,INE090A01021,8.053466,9.734319,0.000000,1879722.0,0.000000,44.0,26.153252,NaN,136327.09,161336.47
...,...,...,...,...,...,...,...,...,...,...,...,...
4606,SHVSUIT,INE02Z901011,8.270599,-6.749615,31.617878,4176.0,0.000000,NaN,NaN,0.00,-0.01,3.73
4607,EVERTEX,INE229N01010,NaN,NaN,74.009687,3700.0,0.000000,NaN,NaN,0.83,-0.14,0.00
4608,SAGRSOY-B,INE131O01024,NaN,1.031146,9.013138,13464.0,0.000000,NaN,NaN,1.72,0.50,0.60
4609,IGLFXPL-B,INE684U01011,NaN,374.248591,53.960430,17959.0,0.000000,NaN,NaN,13.85,-3.25,71.97


In [10]:
df_AdditionalData

,Ticker,ISIN,5Y Avg ROE,5Y Revenue Growth,Promoter Holding,No. of Shareholders,Pledged Promoter Holdings,Rating agency Buy Reco,Volatility,Total Debt,Free Cash Flow,Total Revenue(For last QTR)
0,RELIANCE,INE002A01018,10.627669,12.027323,50.614606,3306662.0,0.000000,21.0,25.405350,270648.00,-79652.00,505153.00
1,TCS,INE467B01029,36.796971,9.886696,72.297047,2474310.0,0.345256,22.0,21.197738,7818.00,36969.00,195772.00
2,HDFCBANK,INE040A01034,17.295561,14.242193,25.780743,2151630.0,0.000000,35.0,25.635825,NaN,40780.31,167695.40
3,INFY,INE009A01021,25.828780,11.618708,13.114253,2128827.0,0.000000,36.0,23.936476,5474.00,21724.00,123936.00
4,ICICIBANK,INE090A01021,8.053466,9.734319,0.000000,1879722.0,0.000000,44.0,26.153252,NaN,136327.09,161336.47
...,...,...,...,...,...,...,...,...,...,...,...,...
4606,SHVSUIT,INE02Z901011,8.270599,-6.749615,31.617878,4176.0,0.000000,NaN,NaN,0.00,-0.01,3.73
4607,EVERTEX,INE229N01010,NaN,NaN,74.009687,3700.0,0.000000,NaN,NaN,0.83,-0.14,0.00
4608,SAGRSOY-B,INE131O01024,NaN,1.031146,9.013138,13464.0,0.000000,NaN,NaN,1.72,0.50,0.60
4609,IGLFXPL-B,INE684U01011,NaN,374.248591,53.960430,17959.0,0.000000,NaN,NaN,13.85,-3.25,71.97


In [11]:
# Merge the Additional metrics data with the merged data frame ( seed and Web)
df_WebAndMoreData = pd.merge(df_mergedWebData, df_AdditionalData[
    ['ISIN',
     'Ticker',
     '5Y Avg ROE',
     '5Y Revenue Growth',
     'Promoter Holding',
     'No. of Shareholders',
     'Pledged Promoter Holdings',
     'Rating agency Buy Reco',
     'Volatility', 
     'Total Debt']], on = 'ISIN', how = 'outer', indicator='true')

In [12]:
df_WebAndMoreData[df_WebAndMoreData['Company'].duplicated()].groupby(df_WebAndMoreData['Company'])

In [13]:
df_WebAndMoreData.to_excel("Merged Social & Additional Data.xlsx")

C:\Users\amshuk\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1168: UserWarning: Ignoring URL 'https://twitter.com/abbottnews,https://www.facebook.com/AbbottCareers,http://www.linkedin.com/company/1612,www.youtube.com/user/,http://abbvie.com,http://www.computershare.com/investor,http://www.computershare.com/us/Pages/default.aspx,http://www.senate.gov/legislative/Public_Disclosure/LDA_reports.htm,http://disclosures.house.gov/ld/ldsearch.aspx,http://online.barrons.com/article/SB50001424052748704836204578360522421954096.html' with link or location/anchor > 2079 characters since it exceeds Excel's limit for URLS
  warn("Ignoring URL '%s' with link or location/anchor > %d "
